In [ ]:
#My process for this question is divided into three parts. 
# (1) I parse the data and format it correctly
# (2) I initalize and fit the decision tree
# (3) I print out my results so that I can complete the decision rules analysis part of the question

In [ ]:
import pandas as pd
import os
import numpy as np
import sklearn
import copy
import csv

In [ ]:
from sklearn.tree import DecisionTreeClassifier 
from matplotlib import pyplot 

In [ ]:
## Reading from a csv file
traindata = []
with open('trainingdata', 'r') as csvfile:  #CHANGE PATH TO WHERE YOU HAVE DATA
    reader = csv.reader(csvfile)
    for row in reader:
        traindata.append(row)
        
validdata = []
with open('validationdata', 'r') as csvfile: #CHANGE PATH TO WHERE YOU HAVE DATA
    reader = csv.reader(csvfile)
    for row in reader:
        validdata.append(row)
        
testdata = []
with open('testdata', 'r') as csvfile: #CHANGE PATH TO WHERE YOU HAVE DATA
    reader = csv.reader(csvfile)
    for row in reader:
        testdata.append(row)

In [ ]:
# Feature names for each column
feature_names = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 
                 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 
                 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 
                 'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'count', 
                 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 
                 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 
                 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_name_src_port_rate',
                 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 
                 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate']

column_names = feature_names + ["attack"]

print("Column names are: ", column_names)

In [ ]:
###Convert the all the values into numeric

### Convert the read data into dataframe
traindata_df = pd.DataFrame(traindata, columns=column_names)
#print(traindata)
#print("Number of Instances: %d  \
#       \nNumber of Features %d " %(len(df), len(feature_names)))

validdata_df = pd.DataFrame(validdata, columns=column_names)
#print(validdata)
#print("Number of Instances: %d  \
#       \nNumber of Features %d " %(len(df), len(feature_names)))

testdata_df = pd.DataFrame(testdata, columns=feature_names)
#print(testdata)
#print("Number of Instances: %d  \
#       \nNumber of Features %d " %(len(df), len(feature_names)))

In [ ]:
#Was geting an error for icmp not being a string value. Solved by the following to convert my data sets into the right format to be able to work with
for name in feature_names:
    if not (any(char.isalpha() for char in traindata_df[name][0])): continue
    
    names = set()
    for element in traindata_df[name]:
        names.add(element)
        
    for element in validdata_df[name]:
        names.add(element)
        
    for element in testdata_df[name]:
        names.add(element)
        
    names= list(names)
    
    for i in range(len(traindata_df[name])):
        traindata_df[name][i] = names.index(traindata_df[name][i])
    
    for i in range(len(validdata_df[name])):
        validdata_df[name][i] = names.index(validdata_df[name][i])
    
    for i in range(len(testdata_df[name])):
        testdata_df[name][i] = names.index(testdata_df[name][i])

names = set()
for element in traindata_df['attack']:
    names.add(element)
        
for element in validdata_df['attack']:
    names.add(element)
    
class_names = list(names)
for i in range(len(traindata_df['attack'])):
    traindata_df['attack'][i] = class_names.index(traindata_df['attack'][i])
    
for i in range(len(validdata_df['attack'])):
    validdata_df['attack'][i] = class_names.index(validdata_df['attack'][i])

In [ ]:
## choosing the feat and targets 
Xtrain, Ytrain = traindata_df[feature_names], traindata_df['attack']
Xvalid, Yvalid = validdata_df[feature_names], validdata_df['attack']
Xtest = testdata_df[feature_names]

Xtrain = Xtrain.astype('float')
Ytrain = Ytrain.astype('float')

Xvalid = Xvalid.astype('float')
Yvalid = Yvalid.astype('float')

Xtest = Xtest.astype('float')

In [ ]:
# import DecisionTree classifier from library
from sklearn.tree import DecisionTreeClassifier

In [ ]:
## Define the model 
dtree = DecisionTreeClassifier(random_state = 0)

In [ ]:
## Training the model 
dtree.fit(Xtrain, Ytrain)

In [ ]:
def accuracy_eq(Ypred,Yother):
    same = 0
    for i in range(len(Yother)):
        if (Yother[i] == Ypred[i]):
            same += 1
            
    return same/len(Yother)

# check train results
Ypred    = dtree.predict(Xtrain)
accuracy = accuracy_eq(list(Ypred),list(Ytrain))
print("Training Accuracy: ", accuracy)
    
# check valid results
Ypred    = dtree.predict(Xvalid)
accuracy = accuracy_eq(list(Ypred),list(Yvalid))
print("Validation Accuracy: ", accuracy)

# calculate classes for test dataset
Ypred = dtree.predict(Xtest)
Ypred = list(Ypred)

In [ ]:
# Get the actual names for the results
for i in range(len(Ypred)):
    for j in range(len(class_names)):
        if (j==int(Ypred[i])):
            Ypred[i] = class_names[j]
            break

with open("results",'w') as result_file: #CHANGE PATH TO WHERE YOU WANT YOUR RESULTS
    for line in Ypred:
        result_file.write(line + "\n")